In [ ]:
# Imports
import requests
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from bs4 import BeautifulSoup
import plotly.io as pio
from sklearn.covariance import LedoitWolf
import re

***RIESGO PAIS ARGENTINA***

In [ ]:
# Riesgo País
prima_riesgo_mas_reciente = None
fecha_actualizacion = None
fuente_datos = None

hoy = datetime.now()
hace_dos_anios = hoy - timedelta(days=730)
fecha_inicio = hace_dos_anios.strftime('%d-%m-%Y')
fecha_fin = hoy.strftime('%d-%m-%Y')

url_ambito = f"https://mercados.ambito.com/riesgopais/historico-general/{fecha_inicio}/{fecha_fin}"

headers_completos = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://mercados.ambito.com/',
    'Connection': 'keep-alive',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin'
}

try:
    session = requests.Session()
    response = session.get(url_ambito, headers=headers_completos, timeout=15)
    
    if response.status_code == 200:
        datos = response.json()
        df = pd.DataFrame(datos[1:], columns=datos[0])
        df['Puntos'] = df['Puntos'].astype(str).str.replace(',', '.')
        df['Puntos'] = pd.to_numeric(df['Puntos'])
        df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d-%m-%Y')
        df = df.sort_values('Fecha', ascending=False).reset_index(drop=True)
        
        prima_riesgo_mas_reciente = df.iloc[0]['Puntos'] / 100
        fecha_actualizacion = df.iloc[0]['Fecha'].strftime('%d/%m/%Y')
        fuente_datos = "Ámbito Financiero"
    else:
        print(f"⚠️ Ámbito respondió con código {response.status_code}. Intentando Rava...")
except Exception as e:
    print(f"⚠️ Error en Ámbito: {str(e)}. Intentando Rava...")

if prima_riesgo_mas_reciente is None:
    try:
        print("🔄 Intentando Rava...")
        url_rava = "https://www.rava.com/perfil/riesgo%20pais"
        
        headers_rava = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'es-AR,es;q=0.9,en;q=0.8',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }
        
        session_rava = requests.Session()
        response_rava = session_rava.get(url_rava, headers=headers_rava, timeout=15, allow_redirects=True)
        
        if response_rava.status_code == 200:
            soup = BeautifulSoup(response_rava.content, 'html.parser')
            all_text = soup.get_text()
            valor_encontrado = None
            
            price_elem = soup.find('div', class_='price')
            if price_elem:
                texto_price = price_elem.get_text(strip=True)
                numeros_solo = re.sub(r'[^\d,\.]', '', texto_price)
                match = re.search(r'(\d{3,4})[,\.]?(\d{2})?', numeros_solo)
                if match:
                    valor_encontrado = match.group(1) + '.' + (match.group(2) if match.group(2) else '00')
            
            if not valor_encontrado:
                matches = re.findall(r'(\d{3,4})[,\.](\d{2})', all_text)
                if matches:
                    for match in matches:
                        valor_num = float(match[0] + '.' + match[1])
                        if 300 <= valor_num <= 3000:
                            valor_encontrado = match[0] + '.' + match[1]
                            break
            
            if valor_encontrado:
                prima_riesgo_mas_reciente = float(valor_encontrado) / 100
                fecha_actualizacion = datetime.now().strftime('%d/%m/%Y')
                fuente_datos = "Rava Bursátil"
        else:
            print(f"⚠️ Rava respondió con código {response_rava.status_code}")
    except Exception as e:
        print(f"❌ Error en Rava: {str(e)}")

if prima_riesgo_mas_reciente is None:
    print("\n" + "="*70)
    print("❌ ERROR: No se pudo obtener el Riesgo País automáticamente.")
    print("="*70)
    print("\n💡 SOLUCIÓN: Ingresa el valor manualmente desde alguna de estas fuentes:")
    print("   📍 https://www.ambito.com/contenidos/riesgo-pais.html")
    print("   📍 https://www.rava.com/perfil/riesgo%20pais")
    print("\n")
    
    try:
        valor_manual = input("Ingresa el Riesgo País en puntos básicos (ej: 650): ")
        if valor_manual and valor_manual.strip():
            valor_limpio = re.sub(r'[^\d\.]', '', valor_manual)
            if valor_limpio:
                valor_puntos = float(valor_limpio)
                # Convertir puntos básicos a porcentaje: 650 puntos = 6.5%
                prima_riesgo_mas_reciente = valor_puntos / 100
                
                fecha_actualizacion = datetime.now().strftime('%d/%m/%Y')
                fuente_datos = "Ingreso Manual"
    except Exception as e:
        print(f"Error al procesar input: {str(e)}")

if prima_riesgo_mas_reciente is None:
    raise RuntimeError("❌ ERROR CRÍTICO: No se pudo obtener el Riesgo País. Por favor, ejecuta la celda nuevamente e ingresa el valor manualmente.")

if prima_riesgo_mas_reciente is not None:
    html = f"""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 20px;
        padding: 40px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        text-align: center;
        max-width: 500px;
        margin: 30px auto;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    ">
        <h2 style="
            color: white;
            font-size: 24px;
            margin-bottom: 10px;
            font-weight: 300;
            letter-spacing: 1px;
        ">Prima de Riesgo País</h2>

        <div style="
            background: rgba(255,255,255,0.95);
            border-radius: 15px;
            padding: 30px;
            margin: 20px 0;
        ">
            <div style="
                font-size: 64px;
                font-weight: bold;
                color: #667eea;
                margin-bottom: 10px;
            ">{prima_riesgo_mas_reciente:.2f}%</div>

            <div style="
                font-size: 14px;
                color: #666;
                margin-top: 15px;
            ">Actualizado: {fecha_actualizacion}</div>
        </div>

        <div style="
            color: rgba(255,255,255,0.8);
            font-size: 12px;
            margin-top: 15px;
        ">Fuente: {fuente_datos}</div>
    </div>
    """

    display(HTML(html))
else:
    print("❌ Error crítico: No se pudo obtener el riesgo país de ninguna fuente.")

***EMPRESAS***

In [ ]:
# Empresas
activos_portfolio = []

banderas_paises = {
    'United States': '🇺🇸', 'Argentina': '🇦🇷', 'Brazil': '🇧🇷', 'Mexico': '🇲🇽',
    'Chile': '🇨🇱', 'Colombia': '🇨🇴', 'Peru': '🇵🇪', 'United Kingdom': '🇬🇧',
    'Germany': '🇩🇪', 'France': '🇫🇷', 'Spain': '🇪🇸', 'Italy': '🇮🇹',
    'Japan': '🇯🇵', 'China': '🇨🇳', 'South Korea': '🇰🇷', 'Canada': '🇨🇦',
    'Australia': '🇦🇺', 'India': '🇮🇳', 'Netherlands': '🇳🇱', 'Switzerland': '🇨🇭',
    'US': '🇺🇸', 'AR': '🇦🇷', 'BR': '🇧🇷', 'MX': '🇲🇽', 'CL': '🇨🇱',
    'CO': '🇨🇴', 'PE': '🇵🇪', 'GB': '🇬🇧', 'DE': '🇩🇪', 'FR': '🇫🇷',
    'ES': '🇪🇸', 'IT': '🇮🇹', 'JP': '🇯🇵', 'CN': '🇨🇳', 'KR': '🇰🇷',
    'CA': '🇨🇦', 'AU': '🇦🇺', 'IN': '🇮🇳', 'NL': '🇳🇱', 'CH': '🇨🇭',
    'Belgium': '🇧🇪', 'Sweden': '🇸🇪', 'Norway': '🇳🇴', 'Denmark': '🇩🇰',
    'Finland': '🇫🇮', 'Ireland': '🇮🇪', 'Portugal': '🇵🇹', 'Austria': '🇦🇹',
    'Israel': '🇮🇱', 'Singapore': '🇸🇬', 'Hong Kong': '🇭🇰', 'Taiwan': '🇹🇼',
    'Thailand': '🇹🇭', 'Malaysia': '🇲🇾', 'Indonesia': '🇮🇩', 'Philippines': '🇵🇭',
    'New Zealand': '🇳🇿', 'South Africa': '🇿🇦', 'Russia': '🇷🇺', 'Turkey': '🇹🇷'
}

output_header = widgets. Output()
output_validacion = widgets.Output()
output_lista = widgets.Output()

input_ticker = widgets.Text(
    placeholder='Ej: AAPL, GGAL, TSLA',
    description='',
    style={'description_width': '0px'},
    layout=widgets.Layout(width='300px', height='45px')
)

btn_agregar = widgets.Button(
    description='Agregar',
    button_style='primary',
    layout=widgets.Layout(width='120px', height='45px'),
    style={'button_color': '#667eea', 'font_weight': 'bold'}
)

btn_finalizar = widgets. Button(
    description='Finalizar Portfolio',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px', margin='20px 0 0 0'),
    style={'font_weight': 'bold'}
)

def obtener_info_activo(ticker):
    try:
        activo = yf.Ticker(ticker)
        info = activo.info

        nombre = info.get('longName') or info.get('shortName') or ticker
        pais = info.get('country', 'United States')
        emoji_bandera = banderas_paises.get(pais, '🌐')

        return {
            'ticker': ticker. upper(),
            'nombre': nombre,
            'pais': pais,
            'bandera': emoji_bandera,
            'valido': True
        }
    except:
        return {'valido': False}

def actualizar_lista():
    with output_lista:
        clear_output()
        if activos_portfolio:
            html_items = ""
            for i, activo in enumerate(activos_portfolio):
                html_items += f"""
                <div style="
                    background: white;
                    border-left: 4px solid #667eea;
                    padding: 15px 20px;
                    margin: 10px 0;
                    border-radius: 8px;
                    box-shadow: 0 2px 8px rgba(0,0,0,0. 1);
                    display: flex;
                    align-items: center;
                    justify-content: space-between;
                ">
                    <div style="display: flex; align-items: center; gap: 15px;">
                        <span style="font-size: 32px;">{activo['bandera']}</span>
                        <div>
                            <div style="font-weight: bold; font-size: 18px; color: #333;">
                                {activo['ticker']}
                            </div>
                            <div style="color: #666; font-size: 14px;">
                                {activo['nombre']}
                            </div>
                            <div style="color: #999; font-size: 12px; margin-top: 3px;">
                                {activo['pais']}
                            </div>
                        </div>
                    </div>
                    <div style="
                        background: #667eea;
                        color: white;
                        padding: 5px 12px;
                        border-radius: 20px;
                        font-size: 12px;
                        font-weight: bold;
                    ">#{i+1}</div>
                </div>
                """

            display(HTML(f"""
            <div style="max-width: 600px; margin: 20px auto;">
                <h3 style="color: #667eea; text-align: center; margin-bottom: 20px;">
                    📊 Activos en el Portfolio ({len(activos_portfolio)})
                </h3>
                {html_items}
            </div>
            """))

def on_agregar_click(b):
    ticker = input_ticker. value.strip().upper()

    with output_validacion:
        clear_output()

        if not ticker:
            display(HTML("""
            <div style="color: #e74c3c; padding: 10px; text-align: center;">
                ⚠️ Por favor ingresa un ticker
            </div>
            """))
            return

        if any(a['ticker'] == ticker for a in activos_portfolio):
            display(HTML(f"""
            <div style="color: #e74c3c; padding: 10px; text-align: center;">
                ⚠️ {ticker} ya está en el portfolio
            </div>
            """))
            return

        display(HTML("""
        <div style="color: #667eea; padding: 10px; text-align: center;">
            🔍 Validando ticker...
        </div>
        """))

        info = obtener_info_activo(ticker)

        clear_output()

        if info['valido']:
            activos_portfolio.append(info)
            display(HTML(f"""
            <div style="
                background: #d4edda;
                color: #155724;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                border: 1px solid #c3e6cb;
            ">
                ✅ {info['bandera']} <strong>{info['ticker']}</strong> - {info['nombre']} agregado exitosamente
            </div>
            """))
            input_ticker. value = ''
            actualizar_lista()
        else:
            display(HTML(f"""
            <div style="
                background: #f8d7da;
                color: #721c24;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                border: 1px solid #f5c6cb;
            ">
                ❌ No se pudo validar el ticker <strong>{ticker}</strong>. Verifica que sea correcto.
            </div>
            """))

def on_finalizar_click(b):
    with output_validacion:
        clear_output()
        if len(activos_portfolio) < 2:
            display(HTML("""
            <div style="
                background: #fff3cd;
                color: #856404;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                border: 1px solid #ffeaa7;
            ">
                ⚠️ Necesitas al menos 2 activos para crear un portfolio
            </div>
            """))
        else:
            display(HTML(f"""
            <div style="
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                padding: 30px;
                border-radius: 15px;
                text-align: center;
                box-shadow: 0 10px 30px rgba(0,0,0,0.3);
            ">
                <h2 style="margin: 0 0 10px 0;">🎉 Portfolio Configurado</h2>
                <p style="font-size: 18px; margin: 0;">{len(activos_portfolio)} activos listos para análisis</p>
            </div>
            """))

btn_agregar.on_click(on_agregar_click)
btn_finalizar. on_click(on_finalizar_click)

with output_header:
    display(HTML("""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 30px;
        border-radius: 15px;
        text-align: center;
        margin-bottom: 30px;
        box-shadow: 0 10px 30px rgba(0,0,0,0. 2);
    ">
        <h1 style="color: white; margin: 0 0 10px 0; font-size: 32px;">
            📈 Configuración de Portfolio
        </h1>
        <p style="color: rgba(255,255,255,0. 9); margin: 0; font-size: 16px;">
            Ingresa los tickers de los activos que conformarán tu portfolio
        </p>
    </div>
    """))

input_box = widgets.HBox(
    [input_ticker, btn_agregar],
    layout=widgets.Layout(
        justify_content='center',
        margin='20px 0',
        gap='10px'
    )
)

container = widgets.VBox([
    output_header,
    input_box,
    output_validacion,
    output_lista,
    widgets.HBox([btn_finalizar], layout=widgets.Layout(justify_content='center'))
])

display(container)

***FRONTERA EFICIENTE***

In [ ]:
# Frontera Eficiente
try:
    _ = activos_portfolio
    _ = prima_riesgo_mas_reciente
except NameError:
    print("❌ Error: Las variables 'activos_portfolio' o 'prima_riesgo_mas_reciente' no están definidas.")
    raise NameError("Faltan variables de configuración.")

retornos_esperados_manuales = {}

pio.renderers.default = 'notebook'

def obtener_tasa_libre_riesgo():
    try:
        tnx = yf.Ticker('^TNX')
        datos = tnx.history(period='5d')
        if not datos.empty: return datos['Close'].iloc[-1] / 100
    except: pass

    try:
        fvx = yf.Ticker('^FVX')
        datos = fvx.history(period='5d')
        if not datos.empty: return datos['Close'].iloc[-1] / 100
    except: pass

    try:
        url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield"
        r = requests.get(url, timeout=5)
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('table', {'class': 't-chart'})
        if table:
            rows = table.find_all('tr')
            if len(rows) > 1:
                cols = rows[-1].find_all('td')
                if len(cols) > 10: return float(cols[10].text.strip()) / 100
    except: pass

    return 0.042

def descargar_precios_historicos(tickers, periodo='10y'):
    if not tickers: return pd.DataFrame()
    datos = yf.download(tickers, period=periodo, progress=False, threads=True)

    if isinstance(datos.columns, pd.MultiIndex):
        precios = datos['Adj Close'] if 'Adj Close' in datos else datos['Close']
    else:
        precios = datos['Adj Close'] if 'Adj Close' in datos else datos['Close']

    if isinstance(precios, pd.Series):
        precios = precios.to_frame(name=tickers[0])

    if precios.index.tz is not None:
        precios.index = precios.index.tz_localize(None)

    return precios.dropna()

def obtener_datos_mercado_sp500():
    end = datetime.now()
    start = end - timedelta(days=3650)
    sp500 = yf.Ticker('^GSPC').history(start=start, end=end)['Close']
    if sp500.index.tz is not None:
        sp500.index = sp500.index.tz_localize(None)
    return sp500

def calcular_retornos_hibridos(precios, manuales, tasa_libre_riesgo, lista_activos):
    """
    Calcula retornos esperados híbridos (CAPM + Manual) y siempre guarda el CAPM teórico.
    Retorna: (retornos_finales, retornos_capm_teoricos)
    """
    sp500 = obtener_datos_mercado_sp500()

    precios_sem = precios.resample('W').last()
    sp500_sem = sp500.resample('W').last()

    ret_activos_sem = precios_sem.pct_change().dropna()
    ret_sp500_sem = sp500_sem.pct_change().dropna()

    datos_sync = pd.concat([ret_activos_sem, ret_sp500_sem], axis=1, join='inner').dropna()

    if datos_sync.empty:
        ret_activos_final = ret_activos_sem
        ret_sp500_final = pd.Series()
    else:
        ret_activos_final = datos_sync.iloc[:, :-1]
        ret_sp500_final = datos_sync.iloc[:, -1]

    prima_mercado = 0.055
    retornos_finales = pd.Series(index=precios.columns, dtype='float64')
    retornos_capm_teoricos = pd.Series(index=precios.columns, dtype='float64')
    pais_map = {a['ticker']: a['pais'] for a in lista_activos}

    try:
        riesgo_pais_arg = float(prima_riesgo_mas_reciente) / 10000
    except:
        riesgo_pais_arg = 0.0

    for ticker in precios.columns:
        try:
            if not ret_sp500_final.empty and ticker in ret_activos_final.columns:
                cov = np.cov(ret_activos_final[ticker], ret_sp500_final)[0][1]
                var_mercado = np.var(ret_sp500_final, ddof=1)
                beta = cov / var_mercado if var_mercado > 0 else 1.0
            else:
                beta = 1.0

            capm = tasa_libre_riesgo + (beta * prima_mercado)

            if pais_map.get(ticker) == 'Argentina':
                capm += riesgo_pais_arg

            retornos_capm_teoricos[ticker] = capm
        except:
            retornos_capm_teoricos[ticker] = tasa_libre_riesgo

        if ticker in manuales and manuales[ticker] is not None:
            retorno_manual_total = manuales[ticker] / 100
            if retorno_manual_total != 0:
                retorno_anualizado = ((1 + retorno_manual_total) ** (1/5)) - 1
                retornos_finales[ticker] = retorno_anualizado
                continue

        retornos_finales[ticker] = retornos_capm_teoricos[ticker]

    return retornos_finales, retornos_capm_teoricos

def calcular_matriz_covarianza(precios):
    """
    Calcula la matriz de covarianza usando Ledoit-Wolf Shrinkage 
    para reducir error de estimación y mejorar estabilidad.
    """
    retornos_diarios = precios.pct_change().dropna()
    
    lw = LedoitWolf()
    cov_shrunk = lw.fit(retornos_diarios).covariance_
    
    return cov_shrunk * 252

def calcular_cvar_historico(pesos, retornos_diarios, alpha=0.95):
    """
    Calcula el CVaR (Conditional Value at Risk) histórico al nivel alpha.
    Retorna el CVaR anualizado (valor positivo indica pérdida esperada).
    """
    ret_portfolio = retornos_diarios.dot(pesos)
    var_percentil = np.percentile(ret_portfolio, (1 - alpha) * 100)
    cvar = ret_portfolio[ret_portfolio <= var_percentil].mean()
    return -cvar * np.sqrt(252)

def metricas_portfolio(pesos, ret_esp, cov, rf):
    ret = np.dot(pesos, ret_esp)
    vol = np.sqrt(np.dot(pesos.T, np.dot(cov, pesos)))
    sharpe = (ret - rf) / vol if vol > 0 else 0
    return ret, vol, sharpe

def optimizar_portfolios(ret_esp, cov, rf, retornos_diarios):
    """
    Optimiza 3 portfolios:
    1. Máximo Sharpe
    2. Mínima Varianza
    3. Mínimo CVaR (95%)
    """
    n = len(ret_esp)
    bounds = tuple((0, 1) for _ in range(n))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    init = n * [1./n]

    opt_var = minimize(
        lambda w: np.sqrt(np.dot(w.T, np.dot(cov, w))),
        init, method='SLSQP', bounds=bounds, constraints=constraints
    )

    def neg_sharpe(w):
        r, v, s = metricas_portfolio(w, ret_esp, cov, rf)
        return -s

    opt_sharpe = minimize(
        neg_sharpe, init, method='SLSQP', bounds=bounds, constraints=constraints
    )

    def objetivo_cvar(w):
        return calcular_cvar_historico(w, retornos_diarios, alpha=0.95)

    opt_cvar = minimize(
        objetivo_cvar, init, method='SLSQP', bounds=bounds, constraints=constraints
    )

    return opt_sharpe.x, opt_var.x, opt_cvar.x

def calcular_frontera(ret_esp, cov, ret_min_var, ret_max_asset):
    n = len(ret_esp)
    bounds = tuple((0, 1) for _ in range(n))

    target_returns = np.linspace(ret_min_var, ret_max_asset, 50)
    frontier_vol = []
    frontier_ret = []

    for r in target_returns:
        cons = [
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
            {'type': 'eq', 'fun': lambda x: np.dot(x, ret_esp) - r}
        ]
        res = minimize(
            lambda w: np.sqrt(np.dot(w.T, np.dot(cov, w))),
            n * [1./n], method='SLSQP', bounds=bounds, constraints=cons
        )
        if res.success:
            frontier_vol.append(res.fun)
            frontier_ret.append(r)

    return np.array(frontier_ret), np.array(frontier_vol)

def analizar_portfolio_ui():
    if not activos_portfolio:
        print("❌ No hay activos en el portfolio.")
        return

    style_header = "background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 15px; border-radius: 10px; text-align: center; font-size: 20px; font-weight: bold;"
    display(widgets.HTML(f"<div style='{style_header}'>📊 Asignar Upside Fundamental</div>"))
    display(widgets.HTML("<div style='padding:10px; background:#e3f2fd; color:#1565c0; border-radius:5px; margin:10px 0; text-align:center;'>Ingresa el <b>Upside Total a 5 años (%)</b> para cada activo cargado previamente.<br>Si dejas 0, el modelo usará CAPM.</div>"))

    inputs = {}
    items = []

    for activo in activos_portfolio:
        lbl = widgets.HTML(f"<b>{activo['bandera']} {activo['ticker']}</b>", layout=widgets.Layout(width='150px'))
        val = widgets.FloatText(value=0.0, description='Upside %:', style={'description_width': 'initial'}, layout=widgets.Layout(width='200px'))
        inputs[activo['ticker']] = val
        items.append(widgets.HBox([lbl, val], layout=widgets.Layout(margin='5px', align_items='center')))

    btn_run = widgets.Button(
        description='Ejecutar Optimización',
        layout=widgets.Layout(width='100%', height='50px', margin='20px 0'),
        style={'button_color': '#667eea', 'text_color': 'white', 'font_weight': 'bold'}
    )

    out = widgets.Output()

    def on_click_run(b):
        global tickers, cov, pesos_sharpe, pesos_minvar, pesos_cvar, ret_esp
        
        with out:
            clear_output()
            print("⏳ Ejecutando modelo Quantamental con Ledoit-Wolf y CVaR...")

            for t, widget in inputs.items():
                retornos_esperados_manuales[t] = widget.value

            rf = obtener_tasa_libre_riesgo()
            tickers = [a['ticker'] for a in activos_portfolio]
            precios = descargar_precios_historicos(tickers)

            if precios.empty:
                print("❌ No se pudieron descargar precios.")
                return

            precios = precios.ffill().bfill()
            retornos_diarios = precios.pct_change().dropna()
            
            ret_esp, ret_capm = calcular_retornos_hibridos(precios, retornos_esperados_manuales, rf, activos_portfolio)
            cov = calcular_matriz_covarianza(precios)

            pesos_sharpe, pesos_minvar, pesos_cvar = optimizar_portfolios(ret_esp, cov, rf, retornos_diarios)
            r_s, v_s, s_s = metricas_portfolio(pesos_sharpe, ret_esp, cov, rf)
            r_m, v_m, s_m = metricas_portfolio(pesos_minvar, ret_esp, cov, rf)
            r_c, v_c, s_c = metricas_portfolio(pesos_cvar, ret_esp, cov, rf)
            
            cvar_s = calcular_cvar_historico(pesos_sharpe, retornos_diarios, 0.95)
            cvar_m = calcular_cvar_historico(pesos_minvar, retornos_diarios, 0.95)
            cvar_c = calcular_cvar_historico(pesos_cvar, retornos_diarios, 0.95)

            f_ret, f_vol = calcular_frontera(ret_esp, cov, r_m, ret_esp.max())

            n_sim = 50000
            w_sim = np.random.random((n_sim, len(tickers)))
            w_sim /= w_sim.sum(axis=1)[:, np.newaxis]
            r_sim = np.dot(w_sim, ret_esp)
            v_sim = np.sqrt(np.einsum('ij,jk,ik->i', w_sim, cov, w_sim))
            s_sim = (r_sim - rf) / v_sim
            
            cvar_sim = np.array([calcular_cvar_historico(w, retornos_diarios, 0.95) for w in w_sim[::100]])

            fig = go.Figure()

            fig.add_trace(go.Scatter(
                x=v_sim*100,
                y=r_sim*100,
                mode='markers',
                marker=dict(
                    color=s_sim,
                    colorscale='Viridis',
                    showscale=True,
                    size=4,
                    colorbar=dict(title="Sharpe Ratio")
                ),
                name='Simulación',
                hovertemplate='Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>'
            ))

            fig.add_trace(go.Scatter(
                x=f_vol*100,
                y=f_ret*100,
                mode='lines',
                line=dict(color='red', width=3),
                name='Frontera Eficiente'
            ))

            max_vol_plot = max(v_sim.max(), v_s * 2) * 100
            y_cml_end = (rf + s_s * (max_vol_plot/100)) * 100
            fig.add_shape(
                type='line',
                x0=0, y0=rf*100,
                x1=max_vol_plot, y1=y_cml_end,
                line=dict(color='orange', width=2, dash='dash')
            )
            fig.add_trace(go.Scatter(
                x=[None], y=[None],
                mode='lines',
                line=dict(color='orange', width=2, dash='dash'),
                name='CML'
            ))

            fig.add_trace(go.Scatter(
                x=[v_s*100],
                y=[r_s*100],
                mode='markers',
                marker=dict(size=20, color='gold', symbol='star', line=dict(color='black', width=2)),
                name='Máximo Sharpe',
                hovertemplate=f'<b>Max Sharpe</b><br>Vol: {v_s*100:.2f}%<br>Ret: {r_s*100:.2f}%<br>Sharpe: {s_s:.2f}<br>CVaR: {cvar_s*100:.2f}%<extra></extra>'
            ))

            fig.add_trace(go.Scatter(
                x=[v_m*100],
                y=[r_m*100],
                mode='markers',
                marker=dict(size=18, color='cyan', symbol='diamond', line=dict(color='black', width=2)),
                name='Mínima Varianza',
                hovertemplate=f'<b>Min Varianza</b><br>Vol: {v_m*100:.2f}%<br>Ret: {r_m*100:.2f}%<br>Sharpe: {s_m:.2f}<br>CVaR: {cvar_m*100:.2f}%<extra></extra>'
            ))

            fig.add_trace(go.Scatter(
                x=[v_c*100],
                y=[r_c*100],
                mode='markers',
                marker=dict(size=18, color='purple', symbol='hexagon', line=dict(color='black', width=2)),
                name='Mínimo CVaR',
                hovertemplate=f'<b>Mínimo CVaR</b><br>Vol: {v_c*100:.2f}%<br>Ret: {r_c*100:.2f}%<br>Sharpe: {s_c:.2f}<br>CVaR: {cvar_c*100:.2f}%<extra></extra>'
            ))

            ind_vol = np.sqrt(np.diag(cov)) * 100
            ind_ret = ret_esp * 100
            fig.add_trace(go.Scatter(
                x=ind_vol,
                y=ind_ret,
                mode='markers+text',
                marker=dict(size=12, color='white', line=dict(color='blue', width=2)),
                text=tickers,
                textposition='top center',
                textfont=dict(size=10, color='black'),
                name='Activos',
                hovertemplate='<b>%{text}</b><br>Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>'
            ))

            fig.update_layout(
                title='<b>Optimización Quantamental con Ledoit-Wolf Shrinkage & CVaR</b>',
                xaxis_title='Riesgo (Volatilidad Anual %)',
                yaxis_title='Retorno Esperado (Anual %)',
                template='plotly_white',
                height=700,
                showlegend=True,
                legend=dict(
                    orientation="v",
                    yanchor="top",
                    y=0.99,
                    xanchor="right",
                    x=0.99,
                    bgcolor="rgba(255, 255, 255, 0.9)",
                    bordercolor="rgba(0, 0, 0, 0.2)",
                    borderwidth=1
                )
            )

            display(HTML(f"""
            <div style="display:flex; gap:15px; justify-content:center; margin-bottom:20px; flex-wrap: wrap;">
                <div style="background:#fff8e1; padding:15px; border-radius:10px; border-left:5px solid #ffd700; width:280px;">
                    <h3>⭐ Máximo Sharpe</h3>
                    <p>Ret: <b>{r_s:.1%}</b> | Vol: <b>{v_s:.1%}</b><br>Sharpe: <b>{s_s:.2f}</b> | CVaR: <b>{cvar_s:.1%}</b></p>
                </div>
                <div style="background:#e0f7fa; padding:15px; border-radius:10px; border-left:5px solid #00bcd4; width:280px;">
                    <h3>💎 Mínima Varianza</h3>
                    <p>Ret: <b>{r_m:.1%}</b> | Vol: <b>{v_m:.1%}</b><br>Sharpe: <b>{s_m:.2f}</b> | CVaR: <b>{cvar_m:.1%}</b></p>
                </div>
                <div style="background:#f3e5f5; padding:15px; border-radius:10px; border-left:5px solid #9c27b0; width:280px;">
                    <h3>🛡️ Mínimo CVaR</h3>
                    <p>Ret: <b>{r_c:.1%}</b> | Vol: <b>{v_c:.1%}</b><br>Sharpe: <b>{s_c:.2f}</b> | CVaR: <b>{cvar_c:.1%}</b></p>
                </div>
            </div>
            """))

            fig.show()

            html_pesos = "<div style='margin-top:30px;'><h3 style='text-align:center; color:#2C3E50; margin-bottom:20px;'>📊 Distribución de Pesos y Retornos</h3>"
            html_pesos += "<table style='width:100%; border-collapse:collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1);'>"
            html_pesos += "<thead><tr style='background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); color:white;'>"
            html_pesos += "<th style='padding:15px; text-align:left;'>Activo</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Peso Max Sharpe</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Peso Min Var</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Peso Min CVaR</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Tasa CAPM (Teórica)</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Retorno Esperado (Final)</th>"
            html_pesos += "</tr></thead><tbody>"

            for i, (t, ps, pm, pc, rcapm, rfinal) in enumerate(zip(tickers, pesos_sharpe, pesos_minvar, pesos_cvar, ret_capm, ret_esp)):
                bg_color = '#f8f9fa' if i % 2 == 0 else '#ffffff'
                html_pesos += f"<tr style='background:{bg_color}; border-bottom:1px solid #dee2e6;'>"
                html_pesos += f"<td style='padding:12px; font-weight:bold; color:#2C3E50;'>{t}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#0066CC;'>{ps:.2%}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#006400;'>{pm:.2%}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#9c27b0;'>{pc:.2%}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#FF6B6B;'>{rcapm:.2%}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#8B0000; font-weight:bold;'>{rfinal:.2%}</td>"
                html_pesos += "</tr>"

            html_pesos += "</tbody></table></div>"
            display(HTML(html_pesos))

    btn_run.on_click(on_click_run)

    display(widgets.VBox(items))
    display(btn_run)
    display(out)

analizar_portfolio_ui()

***RENDIMIENTO DEL PORTFOLIO***

In [ ]:
# RENDIMIENTO DEL PORTFOLIO
def analizar_rendimiento_historico():
    if not activos_portfolio:
        print("❌ No hay activos en el portfolio.")
        return
    
    print("⏳ Descargando datos históricos de los últimos 10 años...")
    
    tickers = [a['ticker'] for a in activos_portfolio]
    datos_5y = yf.download(tickers, period='10y', progress=False, threads=True)
    
    if isinstance(datos_5y.columns, pd.MultiIndex):
        precios_5y = datos_5y['Adj Close'] if 'Adj Close' in datos_5y else datos_5y['Close']
    else:
        precios_5y = datos_5y['Adj Close'] if 'Adj Close' in datos_5y else datos_5y['Close']
    
    if isinstance(precios_5y, pd.Series):
        precios_5y = precios_5y.to_frame(name=tickers[0])
    
    precios_5y = precios_5y.ffill().bfill().dropna()
    
    if precios_5y.empty:
        print("❌ No se pudieron descargar datos históricos suficientes.")
        return
    
    precios_norm = (precios_5y / precios_5y.iloc[0]) * 100
    
    rendimientos_acum = ((precios_5y / precios_5y.iloc[0]) - 1) * 100
    
    retornos_diarios = precios_5y.pct_change().dropna()
    retornos_anuales = ((precios_5y.iloc[-1] / precios_5y.iloc[0]) ** (1/10) - 1) * 100
    volatilidad_anual = retornos_diarios.std() * np.sqrt(252) * 100
    
    rf = obtener_tasa_libre_riesgo()
    
    fig = go.Figure()
    
    for ticker in precios_5y.columns:
        fig.add_trace(go.Scatter(
            x=precios_norm.index,
            y=precios_norm[ticker],
            mode='lines',
            name=ticker,
            hovertemplate=f'<b>{ticker}</b><br>Fecha: %{{x}}<br>Valor: %{{y:.2f}}<extra></extra>'
        ))
    
    fig.update_layout(
        title='<b>Rendimiento Histórico del Portfolio (10 años)</b>',
        xaxis_title='Fecha',
        yaxis_title='Valor Normalizado (Base 100)',
        template='plotly_white',
        height=600,
        hovermode='x unified',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="rgba(0, 0, 0, 0.2)",
            borderwidth=1
        )
    )
    
    fig.show()
    
    html_stats = "<div style='margin-top:30px;'><h3 style='text-align:center; color:#2C3E50; margin-bottom:20px;'>📊 Estadísticas de Rendimiento (10 años)</h3>"
    html_stats += "<table style='width:100%; border-collapse:collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1);'>"
    html_stats += "<thead><tr style='background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); color:white;'>"
    html_stats += "<th style='padding:15px; text-align:left;'>Activo</th>"
    html_stats += "<th style='padding:15px; text-align:center;'>Rendimiento Anualizado</th>"
    html_stats += "<th style='padding:15px; text-align:center;'>Rendimiento Total</th>"
    html_stats += "<th style='padding:15px; text-align:center;'>Volatilidad Anual</th>"
    html_stats += f"<th style='padding:15px; text-align:center;'>Sharpe Ratio (rf={rf:.1%})</th>"
    html_stats += "</tr></thead><tbody>"
    
    for i, ticker in enumerate(precios_5y.columns):
        ret_anual = retornos_anuales[ticker]
        ret_total = rendimientos_acum[ticker].iloc[-1]
        vol_anual = volatilidad_anual[ticker]
        sharpe = (ret_anual/100 - rf) / (vol_anual/100) if vol_anual > 0 else 0
        
        color_ret = '#006400' if ret_anual > 0 else '#8B0000'
        bg_color = '#f8f9fa' if i % 2 == 0 else '#ffffff'
        
        html_stats += f"<tr style='background:{bg_color}; border-bottom:1px solid #dee2e6;'>"
        html_stats += f"<td style='padding:12px; font-weight:bold; color:#2C3E50;'>{ticker}</td>"
        html_stats += f"<td style='padding:12px; text-align:center; color:{color_ret}; font-weight:bold;'>{ret_anual:.2f}%</td>"
        html_stats += f"<td style='padding:12px; text-align:center; color:{color_ret};'>{ret_total:.2f}%</td>"
        html_stats += f"<td style='padding:12px; text-align:center; color:#FF6B6B;'>{vol_anual:.2f}%</td>"
        html_stats += f"<td style='padding:12px; text-align:center; color:#667eea;'>{sharpe:.2f}</td>"
        html_stats += "</tr>"
    
    html_stats += "</tbody></table>"
    html_stats += "</tbody></table></div>"
    
    display(HTML(html_stats))
    pesos_eq = np.array([1/len(tickers)] * len(tickers))
    ret_portfolio = retornos_diarios.dot(pesos_eq)
    valor_portfolio = (1 + ret_portfolio).cumprod() * 100
    
    ret_total_pf = ((valor_portfolio.iloc[-1] / 100) - 1) * 100
    ret_anual_pf = ((valor_portfolio.iloc[-1] / 100) ** (1/10) - 1) * 100
    vol_anual_pf = ret_portfolio.std() * np.sqrt(252) * 100
    sharpe_pf = (ret_anual_pf/100 - rf) / (vol_anual_pf/100)
    
    display(HTML(f"""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 15px;
        padding: 30px;
        margin: 30px auto;
        max-width: 700px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
    ">
        <h2 style="color: white; text-align: center; margin-bottom: 25px;">
            📈 Portfolio Equiponderado (10 años)
        </h2>
        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px;">
            <div style="background: rgba(255,255,255,0.95); padding: 20px; border-radius: 10px;">
                <div style="color: #666; font-size: 14px; margin-bottom: 5px;">Rendimiento Total</div>
                <div style="color: {'#006400' if ret_total_pf > 0 else '#8B0000'}; font-size: 32px; font-weight: bold;">
                    {ret_total_pf:+.2f}%
                </div>
            </div>
            <div style="background: rgba(255,255,255,0.95); padding: 20px; border-radius: 10px;">
                <div style="color: #666; font-size: 14px; margin-bottom: 5px;">Rendimiento Anualizado</div>
                <div style="color: {'#006400' if ret_anual_pf > 0 else '#8B0000'}; font-size: 32px; font-weight: bold;">
                    {ret_anual_pf:+.2f}%
                </div>
            </div>
            <div style="background: rgba(255,255,255,0.95); padding: 20px; border-radius: 10px;">
                <div style="color: #666; font-size: 14px; margin-bottom: 5px;">Volatilidad Anual</div>
                <div style="color: #FF6B6B; font-size: 32px; font-weight: bold;">
                    {vol_anual_pf:.2f}%
                </div>
            </div>
            <div style="background: rgba(255,255,255,0.95); padding: 20px; border-radius: 10px;">
                <div style="color: #666; font-size: 14px; margin-bottom: 5px;">Sharpe Ratio</div>
                <div style="color: #667eea; font-size: 32px; font-weight: bold;">
                    {sharpe_pf:.2f}
                </div>
            </div>
        </div>
        <div style="color: rgba(255,255,255,0.8); text-align: center; margin-top: 20px; font-size: 12px;">
            Ponderación: {', '.join([f'{t} ({1/len(tickers):.1%})' for t in tickers])}
        </div>
    </div>
    """))

analizar_rendimiento_historico()

***HEATMAP DE CORRELACIONES***

In [ ]:
# HEATMAP DE CORRELACIONES
def generar_heatmap_correlaciones():
    if not activos_portfolio:
        print("❌ No hay activos en el portfolio.")
        return
    
    print("⏳ Calculando matriz de correlaciones...")
    
    tickers = [a['ticker'] for a in activos_portfolio]
    precios = descargar_precios_historicos(tickers, periodo='10y')
    
    if precios.empty:
        print("❌ No se pudieron descargar los datos históricos.")
        return
    
    precios = precios.ffill().bfill()
    
    retornos_log = np.log(precios / precios.shift(1)).dropna()
    
    matriz_corr = retornos_log.corr()
    
    fig = go.Figure(data=go.Heatmap(
        z=matriz_corr.values,
        x=matriz_corr.columns,
        y=matriz_corr.index,
        colorscale=[
            [0.0, '#0000FF'],    # Azul (correlación -1)
            [0.5, '#FFFFFF'],    # Blanco (correlación 0)
            [1.0, '#FF0000']     # Rojo (correlación +1)
        ],
        zmid=0,
        text=np.round(matriz_corr.values, 2),
        texttemplate='%{text}',
        textfont={"size": 10},
        colorbar=dict(
            title="Correlación",
            titleside="right",
            tickmode="linear",
            tick0=-1,
            dtick=0.5
        ),
        hovertemplate='%{y} vs %{x}<br>Correlación: %{z:.3f}<extra></extra>'
    ))
    
    fig.update_layout(
        title='<b>Matriz de Correlación de Retornos (10 años)</b>',
        xaxis_title='Activos',
        yaxis_title='Activos',
        template='plotly_white',
        height=700,
        width=800,
        xaxis={'side': 'bottom'},
        yaxis={'autorange': 'reversed'}
    )
    
    fig.show()
    
    corr_promedio = matriz_corr.values[np.triu_indices_from(matriz_corr.values, k=1)].mean()
    corr_max = matriz_corr.values[np.triu_indices_from(matriz_corr.values, k=1)].max()
    corr_min = matriz_corr.values[np.triu_indices_from(matriz_corr.values, k=1)].min()
    
    mask = np.triu(np.ones_like(matriz_corr, dtype=bool), k=1)
    corr_masked = matriz_corr.where(mask)
    
    max_corr_idx = np.unravel_index(np.nanargmax(corr_masked.values), corr_masked.shape)
    min_corr_idx = np.unravel_index(np.nanargmin(corr_masked.values), corr_masked.shape)
    
    par_max = f"{matriz_corr.index[max_corr_idx[0]]} - {matriz_corr.columns[max_corr_idx[1]]}"
    par_min = f"{matriz_corr.index[min_corr_idx[0]]} - {matriz_corr.columns[min_corr_idx[1]]}"
    
    display(HTML(f"""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 15px;
        padding: 30px;
        margin: 30px auto;
        max-width: 800px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
    ">
        <h2 style="color: white; text-align: center; margin-bottom: 25px;">
            📊 Análisis de Correlaciones
        </h2>
        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px;">
            <div style="background: rgba(255,255,255,0.95); padding: 20px; border-radius: 10px;">
                <div style="color: #666; font-size: 14px; margin-bottom: 5px;">Correlación Promedio</div>
                <div style="color: #667eea; font-size: 32px; font-weight: bold;">
                    {corr_promedio:.3f}
                </div>
            </div>
            <div style="background: rgba(255,255,255,0.95); padding: 20px; border-radius: 10px;">
                <div style="color: #666; font-size: 14px; margin-bottom: 5px;">Rango de Correlación</div>
                <div style="color: #667eea; font-size: 32px; font-weight: bold;">
                    {corr_min:.3f} a {corr_max:.3f}
                </div>
            </div>
        </div>
        <div style="background: rgba(255,255,255,0.95); padding: 20px; border-radius: 10px; margin-top: 20px;">
            <div style="color: #666; font-size: 14px; margin-bottom: 10px;">Mayor Correlación</div>
            <div style="color: #FF0000; font-size: 20px; font-weight: bold;">
                {par_max}: {corr_max:.3f}
            </div>
        </div>
        <div style="background: rgba(255,255,255,0.95); padding: 20px; border-radius: 10px; margin-top: 20px;">
            <div style="color: #666; font-size: 14px; margin-bottom: 10px;">Menor Correlación (Mejor Diversificación)</div>
            <div style="color: #0000FF; font-size: 20px; font-weight: bold;">
                {par_min}: {corr_min:.3f}
            </div>
        </div>
        <div style="color: rgba(255,255,255,0.9); text-align: center; margin-top: 20px; font-size: 13px; line-height: 1.6;">
            💡 <b>Interpretación:</b><br>
            Valores cercanos a <span style="color: #FF6B6B;">+1</span> indican activos que se mueven juntos (menor diversificación)<br>
            Valores cercanos a <span style="color: #4ECDC4;">-1</span> o <span style="color: #4ECDC4;">0</span> indican mayor potencial de diversificación
        </div>
    </div>
    """))

generar_heatmap_correlaciones()

***UNDERWATER PLOT***

In [ ]:
# UNDERWATER PLOT
def generar_underwater_plot():
    if not activos_portfolio:
        print("❌ No hay activos en el portfolio.")
        return
    
    print("⏳ Calculando drawdowns históricos...")
    
    tickers = [a['ticker'] for a in activos_portfolio]
    precios = descargar_precios_historicos(tickers, periodo='10y')
    
    if precios.empty:
        print("❌ No se pudieron descargar los datos históricos.")
        return
    
    precios = precios.ffill().bfill()
    
    drawdowns = pd.DataFrame(index=precios.index)
    
    for ticker in precios.columns:
        max_acumulado = precios[ticker].cummax()
        drawdowns[ticker] = (precios[ticker] / max_acumulado - 1) * 100
    
    retornos_diarios = precios.pct_change().dropna()
    pesos_eq = np.array([1/len(tickers)] * len(tickers))
    ret_portfolio = retornos_diarios.dot(pesos_eq)
    valor_portfolio = (1 + ret_portfolio).cumprod()
    valor_portfolio = pd.concat([pd.Series([1], index=[precios.index[0]]), valor_portfolio])
    
    max_acum_pf = valor_portfolio.cummax()
    drawdown_portfolio = (valor_portfolio / max_acum_pf - 1) * 100
    
    fig = go.Figure()
    
    for ticker in drawdowns.columns:
        fig.add_trace(go.Scatter(
            x=drawdowns.index,
            y=drawdowns[ticker],
            mode='lines',
            name=ticker,
            fill='tozeroy',
            fillcolor=f'rgba({np.random.randint(50,255)}, {np.random.randint(50,255)}, {np.random.randint(50,255)}, 0.3)',
            line=dict(width=1.5),
            hovertemplate=f'<b>{ticker}</b><br>Fecha: %{{x}}<br>Drawdown: %{{y:.2f}}%<extra></extra>'
        ))
    
    fig.add_trace(go.Scatter(
        x=drawdown_portfolio.index,
        y=drawdown_portfolio.values,
        mode='lines',
        name='Portfolio Equiponderado',
        line=dict(color='black', width=3, dash='solid'),
        fill='tozeroy',
        fillcolor='rgba(100, 100, 100, 0.4)',
        hovertemplate='<b>Portfolio Equiponderado</b><br>Fecha: %{x}<br>Drawdown: %{y:.2f}%<extra></extra>'
    ))
    
    fig.update_layout(
        title='<b>🌊 Profundidad de las Caídas (Drawdown Histórico - 10 años)</b>',
        xaxis_title='Fecha',
        yaxis_title='Drawdown (%)',
        template='plotly_white',
        height=700,
        hovermode='x unified',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99,
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="rgba(0, 0, 0, 0.2)",
            borderwidth=1
        ),
        yaxis=dict(
            ticksuffix='%',
            zeroline=True,
            zerolinewidth=2,
            zerolinecolor='red'
        )
    )
    
    fig.show()
    
    max_dd_por_activo = drawdowns.min()
    max_dd_portfolio = drawdown_portfolio.min()
    
    duraciones = {}
    for ticker in drawdowns.columns:
        dd = drawdowns[ticker]
        en_drawdown = dd < -0.1
        if en_drawdown.any():
            periodos = (en_drawdown != en_drawdown.shift()).cumsum()
            duraciones[ticker] = en_drawdown.groupby(periodos).sum().max()
        else:
            duraciones[ticker] = 0
    
    dd_pf = drawdown_portfolio
    en_drawdown_pf = dd_pf < -0.1
    if en_drawdown_pf.any():
        periodos_pf = (en_drawdown_pf != en_drawdown_pf.shift()).cumsum()
        duracion_pf = en_drawdown_pf.groupby(periodos_pf).sum().max()
    else:
        duracion_pf = 0
    
    html_stats = "<div style='margin-top:30px;'><h3 style='text-align:center; color:#2C3E50; margin-bottom:20px;'>📉 Estadísticas de Drawdown</h3>"
    html_stats += "<table style='width:100%; border-collapse:collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1);'>"
    html_stats += "<thead><tr style='background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); color:white;'>"
    html_stats += "<th style='padding:15px; text-align:left;'>Activo</th>"
    html_stats += "<th style='padding:15px; text-align:center;'>Peor Drawdown</th>"
    html_stats += "<th style='padding:15px; text-align:center;'>Mayor Duración (días)</th>"
    html_stats += "</tr></thead><tbody>"
    
    for i, ticker in enumerate(drawdowns.columns):
        bg_color = '#f8f9fa' if i % 2 == 0 else '#ffffff'
        html_stats += f"<tr style='background:{bg_color}; border-bottom:1px solid #dee2e6;'>"
        html_stats += f"<td style='padding:12px; font-weight:bold; color:#2C3E50;'>{ticker}</td>"
        html_stats += f"<td style='padding:12px; text-align:center; color:#8B0000; font-weight:bold;'>{max_dd_por_activo[ticker]:.2f}%</td>"
        html_stats += f"<td style='padding:12px; text-align:center; color:#FF6B6B;'>{int(duraciones[ticker])} días</td>"
        html_stats += "</tr>"
    
    html_stats += f"<tr style='background:#e3f2fd; border-top:3px solid #667eea; font-weight:bold;'>"
    html_stats += f"<td style='padding:12px; color:#1565c0;'>📊 Portfolio Equiponderado</td>"
    html_stats += f"<td style='padding:12px; text-align:center; color:#8B0000; font-weight:bold;'>{max_dd_portfolio:.2f}%</td>"
    html_stats += f"<td style='padding:12px; text-align:center; color:#FF6B6B;'>{int(duracion_pf)} días</td>"
    html_stats += "</tr>"
    
    html_stats += "</tbody></table></div>"
    
    display(HTML(html_stats))
    
    mejoria_dd = ((max_dd_portfolio - max_dd_por_activo.mean()) / abs(max_dd_por_activo.mean())) * 100
    
    display(HTML(f"""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 15px;
        padding: 30px;
        margin: 30px auto;
        max-width: 800px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
    ">
        <h2 style="color: white; text-align: center; margin-bottom: 25px;">
            🛡️ Efecto Diversificación
        </h2>
        <div style="background: rgba(255,255,255,0.95); padding: 25px; border-radius: 10px;">
            <div style="text-align: center; margin-bottom: 20px;">
                <div style="color: #666; font-size: 14px; margin-bottom: 10px;">
                    Drawdown Promedio Individual vs Portfolio
                </div>
                <div style="display: flex; justify-content: center; gap: 30px; align-items: center;">
                    <div>
                        <div style="color: #999; font-size: 12px;">Promedio Individual</div>
                        <div style="color: #8B0000; font-size: 28px; font-weight: bold;">
                            {max_dd_por_activo.mean():.2f}%
                        </div>
                    </div>
                    <div style="font-size: 32px; color: #667eea;">→</div>
                    <div>
                        <div style="color: #999; font-size: 12px;">Portfolio Diversificado</div>
                        <div style="color: #006400; font-size: 28px; font-weight: bold;">
                            {max_dd_portfolio:.2f}%
                        </div>
                    </div>
                </div>
            </div>
            <div style="text-align: center; padding: 15px; background: #f0f4ff; border-radius: 8px;">
                <div style="color: #667eea; font-size: 16px; font-weight: bold;">
                    {'✅ Mejora de ' + f'{abs(mejoria_dd):.1f}%' if mejoria_dd > 0 else '⚠️ El portfolio no mejora el drawdown promedio'}
                </div>
                <div style="color: #666; font-size: 13px; margin-top: 8px;">
                    La diversificación {'reduce' if mejoria_dd > 0 else 'no reduce significativamente'} la profundidad de las caídas
                </div>
            </div>
        </div>
        <div style="color: rgba(255,255,255,0.9); text-align: center; margin-top: 20px; font-size: 13px; line-height: 1.6;">
            💡 <b>Interpretación:</b><br>
            El Underwater Plot muestra cuánto pierdes desde el máximo histórico en cada momento.<br>
            Un portfolio diversificado suele tener drawdowns menos pronunciados que activos individuales.
        </div>
    </div>
    """))

generar_underwater_plot()

***CONTRIBUCIÓN MARGINAL AL RIESGO***

In [ ]:
# CONTRIBUCIÓN MARGINAL AL RIESGO
def analizar_contribucion_marginal_riesgo():
    if not activos_portfolio:
        print("❌ No hay activos en el portfolio.")
        return
    
    try:
        _ = cov
        _ = pesos_sharpe
        _ = pesos_minvar
        _ = pesos_cvar
        _ = tickers
    except NameError:
        print("❌ Error: Debes ejecutar primero la celda de 'FRONTERA EFICIENTE' para calcular los portfolios óptimos.")
        return
    
    print("⏳ Calculando Contribución Marginal al Riesgo (MCR)...")
    
    def calcular_mcr_y_contribucion(pesos, cov_matrix):
        """
        Calcula la Contribución Marginal al Riesgo (MCR) y la contribución porcentual
        de cada activo al riesgo total del portfolio.
        
        MCR_i = (Σ · w)_i / √(w^T · Σ · w)
        Contribución_i = w_i × MCR_i
        Contribución_%_i = Contribución_i / Volatilidad_Portfolio × 100
        """
        cov_dot_w = np.dot(cov_matrix, pesos)
        
        vol_portfolio = np.sqrt(np.dot(pesos.T, np.dot(cov_matrix, pesos)))
        
        mcr = cov_dot_w / vol_portfolio
        
        contribucion_absoluta = pesos * mcr
        
        contribucion_porcentual = (contribucion_absoluta / vol_portfolio) * 100
        
        return mcr, contribucion_absoluta, contribucion_porcentual, vol_portfolio
    
    mcr_sharpe, contrib_abs_sharpe, contrib_pct_sharpe, vol_sharpe = calcular_mcr_y_contribucion(pesos_sharpe, cov)
    mcr_minvar, contrib_abs_minvar, contrib_pct_minvar, vol_minvar = calcular_mcr_y_contribucion(pesos_minvar, cov)
    mcr_cvar, contrib_abs_cvar, contrib_pct_cvar, vol_cvar = calcular_mcr_y_contribucion(pesos_cvar, cov)
    
    df_mcr = pd.DataFrame({
        'Activo': tickers,
        'Peso Sharpe': pesos_sharpe * 100,
        'MCR Sharpe': mcr_sharpe * 100,
        'Contrib% Sharpe': contrib_pct_sharpe,
        'Peso MinVar': pesos_minvar * 100,
        'MCR MinVar': mcr_minvar * 100,
        'Contrib% MinVar': contrib_pct_minvar,
        'Peso CVaR': pesos_cvar * 100,
        'MCR CVaR': mcr_cvar * 100,
        'Contrib% CVaR': contrib_pct_cvar
    })
    
    np.random.seed(42)  # Para reproducibilidad
    colores = [f'rgb({np.random.randint(50,255)}, {np.random.randint(50,255)}, {np.random.randint(50,255)})' 
               for _ in range(len(tickers))]
    
    fig = go.Figure()
    
    portfolios = ['Máximo Sharpe', 'Mínima Varianza', 'Mínimo CVaR']
    contrib_data = [contrib_pct_sharpe, contrib_pct_minvar, contrib_pct_cvar]
    pesos_data = [pesos_sharpe * 100, pesos_minvar * 100, pesos_cvar * 100]
    vols = [vol_sharpe * 100, vol_minvar * 100, vol_cvar * 100]
    
    for i, ticker in enumerate(tickers):
        contribuciones = [contrib_data[j][i] for j in range(3)]
        pesos = [pesos_data[j][i] for j in range(3)]
        
        fig.add_trace(go.Bar(
            name=ticker,
            x=portfolios,
            y=contribuciones,
            marker_color=colores[i],
            hovertemplate=f'<b>{ticker}</b><br>' +
                         'Portfolio: %{x}<br>' +
                         'Contribución al Riesgo: %{y:.2f}%<br>' +
                         f'Peso: {pesos[0]:.2f}%, {pesos[1]:.2f}%, {pesos[2]:.2f}%<br>' +
                         '<extra></extra>',
            text=[f'{c:.1f}%' if c > 3 else '' for c in contribuciones],
            textposition='inside',
            textfont=dict(color='white', size=10)
        ))
    
    fig.update_layout(
        title='<b>📊 Contribución Marginal al Riesgo por Activo</b><br>' +
              '<sub>Cada barra suma 100% del riesgo total del portfolio</sub>',
        xaxis_title='Portfolio Óptimo',
        yaxis_title='Contribución al Riesgo Total (%)',
        barmode='stack',
        template='plotly_white',
        height=700,
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=1.15,
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="rgba(0, 0, 0, 0.2)",
            borderwidth=1
        ),
        yaxis=dict(range=[0, 100]),
        hovermode='closest'
    )
    
    for i, (portfolio, vol) in enumerate(zip(portfolios, vols)):
        fig.add_annotation(
            x=i,
            y=105,
            text=f'Vol: {vol:.2f}%',
            showarrow=False,
            font=dict(size=11, color='#667eea', family='Arial Black'),
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='#667eea',
            borderwidth=1,
            borderpad=4
        )
    
    fig.show()
    
    html_table = "<div style='margin-top:30px;'><h3 style='text-align:center; color:#2C3E50; margin-bottom:20px;'>📈 Análisis Detallado de Contribución al Riesgo</h3>"
    
    html_table += "<h4 style='color:#667eea; margin-top:30px;'>⭐ Portfolio Máximo Sharpe</h4>"
    html_table += "<table style='width:100%; border-collapse:collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1); margin-bottom:30px;'>"
    html_table += "<thead><tr style='background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); color:white;'>"
    html_table += "<th style='padding:12px; text-align:left;'>Activo</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Peso (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>MCR (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Contribución al Riesgo (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Ratio Riesgo/Peso</th>"
    html_table += "</tr></thead><tbody>"
    
    for i, row in df_mcr.iterrows():
        bg_color = '#f8f9fa' if i % 2 == 0 else '#ffffff'
        ratio = row['Contrib% Sharpe'] / row['Peso Sharpe'] if row['Peso Sharpe'] > 0.01 else 0
        color_ratio = '#006400' if ratio < 1.2 else ('#FF8C00' if ratio < 1.5 else '#8B0000')
        
        html_table += f"<tr style='background:{bg_color}; border-bottom:1px solid #dee2e6;'>"
        html_table += f"<td style='padding:10px; font-weight:bold; color:#2C3E50;'>{row['Activo']}</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#667eea;'>{row['Peso Sharpe']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#FF6B6B;'>{row['MCR Sharpe']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#8B0000; font-weight:bold;'>{row['Contrib% Sharpe']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:{color_ratio}; font-weight:bold;'>{ratio:.2f}x</td>"
        html_table += "</tr>"
    
    html_table += "</tbody></table>"
    
    html_table += "<h4 style='color:#00bcd4; margin-top:30px;'>💎 Portfolio Mínima Varianza</h4>"
    html_table += "<table style='width:100%; border-collapse:collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1); margin-bottom:30px;'>"
    html_table += "<thead><tr style='background:linear-gradient(135deg, #00bcd4 0%, #0097a7 100%); color:white;'>"
    html_table += "<th style='padding:12px; text-align:left;'>Activo</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Peso (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>MCR (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Contribución al Riesgo (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Ratio Riesgo/Peso</th>"
    html_table += "</tr></thead><tbody>"
    
    for i, row in df_mcr.iterrows():
        bg_color = '#f8f9fa' if i % 2 == 0 else '#ffffff'
        ratio = row['Contrib% MinVar'] / row['Peso MinVar'] if row['Peso MinVar'] > 0.01 else 0
        color_ratio = '#006400' if ratio < 1.2 else ('#FF8C00' if ratio < 1.5 else '#8B0000')
        
        html_table += f"<tr style='background:{bg_color}; border-bottom:1px solid #dee2e6;'>"
        html_table += f"<td style='padding:10px; font-weight:bold; color:#2C3E50;'>{row['Activo']}</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#00bcd4;'>{row['Peso MinVar']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#FF6B6B;'>{row['MCR MinVar']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#8B0000; font-weight:bold;'>{row['Contrib% MinVar']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:{color_ratio}; font-weight:bold;'>{ratio:.2f}x</td>"
        html_table += "</tr>"
    
    html_table += "</tbody></table>"
    
    html_table += "<h4 style='color:#9c27b0; margin-top:30px;'>🛡️ Portfolio Mínimo CVaR</h4>"
    html_table += "<table style='width:100%; border-collapse:collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1); margin-bottom:30px;'>"
    html_table += "<thead><tr style='background:linear-gradient(135deg, #9c27b0 0%, #7b1fa2 100%); color:white;'>"
    html_table += "<th style='padding:12px; text-align:left;'>Activo</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Peso (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>MCR (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Contribución al Riesgo (%)</th>"
    html_table += "<th style='padding:12px; text-align:center;'>Ratio Riesgo/Peso</th>"
    html_table += "</tr></thead><tbody>"
    
    for i, row in df_mcr.iterrows():
        bg_color = '#f8f9fa' if i % 2 == 0 else '#ffffff'
        ratio = row['Contrib% CVaR'] / row['Peso CVaR'] if row['Peso CVaR'] > 0.01 else 0
        color_ratio = '#006400' if ratio < 1.2 else ('#FF8C00' if ratio < 1.5 else '#8B0000')
        
        html_table += f"<tr style='background:{bg_color}; border-bottom:1px solid #dee2e6;'>"
        html_table += f"<td style='padding:10px; font-weight:bold; color:#2C3E50;'>{row['Activo']}</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#9c27b0;'>{row['Peso CVaR']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#FF6B6B;'>{row['MCR CVaR']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:#8B0000; font-weight:bold;'>{row['Contrib% CVaR']:.2f}%</td>"
        html_table += f"<td style='padding:10px; text-align:center; color:{color_ratio}; font-weight:bold;'>{ratio:.2f}x</td>"
        html_table += "</tr>"
    
    html_table += "</tbody></table></div>"
    
    display(HTML(html_table))
    
    display(HTML(f"""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 15px;
        padding: 30px;
        margin: 30px auto;
        max-width: 900px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
    ">
        <h2 style="color: white; text-align: center; margin-bottom: 25px;">
            🎯 Interpretación del Análisis MCR
        </h2>
        <div style="background: rgba(255,255,255,0.95); padding: 25px; border-radius: 10px;">
            <h4 style="color: #667eea; margin-top: 0;">📌 ¿Qué es la Contribución Marginal al Riesgo (MCR)?</h4>
            <p style="color: #333; line-height: 1.8; margin: 15px 0;">
                El <b>MCR</b> mide cuánto riesgo adicional aporta cada activo al portfolio total, 
                considerando las correlaciones entre activos. No es lo mismo que la volatilidad individual.
            </p>
            
            <h4 style="color: #667eea; margin-top: 20px;">📊 Ratio Riesgo/Peso</h4>
            <ul style="color: #333; line-height: 1.8;">
                <li><b style="color: #006400;">Ratio < 1.2x:</b> Activo bien diversificado, aporta menos riesgo que su peso</li>
                <li><b style="color: #FF8C00;">Ratio 1.2x - 1.5x:</b> Activo con riesgo proporcional, requiere monitoreo</li>
                <li><b style="color: #8B0000;">Ratio > 1.5x:</b> Activo concentra riesgo, considerar reducir exposición</li>
            </ul>
            
            <h4 style="color: #667eea; margin-top: 20px;">💡 Insights Clave</h4>
            <p style="color: #333; line-height: 1.8; margin: 15px 0;">
                • Un activo con <b>peso pequeño</b> pero <b>alta contribución</b> puede desestabilizar el portfolio<br>
                • El portfolio de <b>Mínima Varianza</b> busca que todos los activos tengan MCR similares<br>
                • El portfolio de <b>Máximo Sharpe</b> tolera mayor concentración de riesgo si mejora el retorno<br>
                • El portfolio de <b>Mínimo CVaR</b> minimiza pérdidas extremas en la cola de distribución
            </p>
        </div>
    </div>
    """))
    
    print("\n✅ Análisis de Contribución Marginal al Riesgo completado.")

analizar_contribucion_marginal_riesgo()